<a href="https://colab.research.google.com/github/SHIVADHARSHINI25/github_emc/blob/main/courses/udacity_intro_to_tensorflow_for_deep_learning/l01c01_introduction_to_colab_and_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2018 The TensorFlow Authors.

In [41]:
import os
from PIL import Image
import scipy.io as sio
import numpy as np
import torch
from torch.utils.data import Dataset
import torchvision.transforms.functional as TF

class CrowdDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform

        self.images = []
        for f in os.listdir(root_dir):
            if f.endswith('.jpg') or f.endswith('.png'):
                base = os.path.splitext(f)[0]
                mat_name = f"GT_{base}.mat"
                mat_path = os.path.join(root_dir, mat_name)
                if os.path.exists(mat_path):
                    self.images.append(f)
                else:
                    print(f"⚠️  Skipping {f} — no matching {mat_name}")

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_name = self.images[idx]
        base_name = os.path.splitext(img_name)[0]
        mat_name = f"GT_{base_name}.mat"

        img_path = os.path.join(self.root_dir, img_name)
        mat_path = os.path.join(self.root_dir, mat_name)

        # Load image
        image = Image.open(img_path).convert('RGB')

        # Load density map from .mat
        mat = sio.loadmat(mat_path)

        # Assumes the .mat contains 'density' key — adjust if needed
        if 'density' in mat:
            density = mat['density']
        else:
            raise KeyError(f"'density' not found in {mat_path}")

        density = density.astype(np.float32)
        density_img = Image.fromarray(density)

        if self.transform:
            image = self.transform(image)
            density_img = TF.resize(density_img, [256, 256])
            density_img = TF.to_tensor(density_img)

        return image, density_img


In [43]:
print(f"Train samples: {len(train_dataset)}")
print(f"Val samples: {len(val_dataset)}")
print(f"Test samples: {len(test_dataset)}")


Train samples: 0
Val samples: 0
Test samples: 0


In [51]:
train_path = '/content/drive/MyDrive/CrowdDataset/train'
val_path   = '/content/drive/MyDrive/CrowdDataset/val'
test_path  = '/content/drive/MyDrive/CrowdDataset/test'


In [52]:
import os

print("Train Files:", os.listdir(train_path))
print("Val Files:", os.listdir(val_path))
print("Test Files:", os.listdir(test_path))


Train Files: ['ground_truth', 'images']
Val Files: ['ground_truth', 'images']
Test Files: ['growth_truth', 'images']


In [53]:
from torch.utils.data import Dataset
import os
from PIL import Image
import scipy.io
import torch

class CrowdDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.images = []

        for f in os.listdir(root_dir):
            if f.endswith('.jpg') or f.endswith('.png'):
                img_path = os.path.join(root_dir, f)
                base_name = os.path.splitext(f)[0]
                mat_name = f"GT_{base_name}.mat"
                mat_path = os.path.join(root_dir, mat_name)

                if os.path.exists(mat_path):
                    self.images.append((img_path, mat_path))
                else:
                    print(f"⚠️ Skipping {f} — no matching {mat_name}")

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path, mat_path = self.images[idx]
        image = Image.open(img_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        mat = scipy.io.loadmat(mat_path)
        # Assume 'image_info' is in .mat (you may need to adapt this!)
        points = mat['image_info'][0][0][0][0][0]
        target = torch.tensor(points.shape[0], dtype=torch.float32)  # crowd count

        return image, target


In [54]:
from torchvision import transforms
from torch.utils.data import DataLoader

transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor()
])

train_dataset = CrowdDataset(train_path, transform=transform)
val_dataset   = CrowdDataset(val_path, transform=transform)
test_dataset  = CrowdDataset(test_path, transform=transform)

# Check the lengths
print(f"Train samples: {len(train_dataset)}")
print(f"Val samples: {len(val_dataset)}")
print(f"Test samples: {len(test_dataset)}")

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=8)
test_loader  = DataLoader(test_dataset, batch_size=8)


Train samples: 0
Val samples: 0
Test samples: 0


ValueError: num_samples should be a positive integer value, but got num_samples=0

In [42]:
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor()
])

train_dataset = CrowdDataset(train_path, transform=transform)
val_dataset = CrowdDataset(val_path, transform=transform)
test_dataset = CrowdDataset(test_path, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8)
test_loader = DataLoader(test_dataset, batch_size=8)

⚠️  Skipping IMG_67.jpg — no matching GT_IMG_67.mat
⚠️  Skipping IMG_65.jpg — no matching GT_IMG_65.mat
⚠️  Skipping IMG_68.jpg — no matching GT_IMG_68.mat
⚠️  Skipping IMG_64.jpg — no matching GT_IMG_64.mat
⚠️  Skipping IMG_61.jpg — no matching GT_IMG_61.mat
⚠️  Skipping IMG_62.jpg — no matching GT_IMG_62.mat
⚠️  Skipping IMG_73.jpg — no matching GT_IMG_73.mat
⚠️  Skipping IMG_74.jpg — no matching GT_IMG_74.mat
⚠️  Skipping IMG_78.jpg — no matching GT_IMG_78.mat
⚠️  Skipping IMG_76.jpg — no matching GT_IMG_76.mat
⚠️  Skipping IMG_77.jpg — no matching GT_IMG_77.mat
⚠️  Skipping IMG_75.jpg — no matching GT_IMG_75.mat
⚠️  Skipping IMG_79.jpg — no matching GT_IMG_79.mat
⚠️  Skipping IMG_80.jpg — no matching GT_IMG_80.mat
⚠️  Skipping IMG_72.jpg — no matching GT_IMG_72.mat
⚠️  Skipping IMG_81.jpg — no matching GT_IMG_81.mat
⚠️  Skipping IMG_63.jpg — no matching GT_IMG_63.mat
⚠️  Skipping IMG_66.jpg — no matching GT_IMG_66.mat
⚠️  Skipping IMG_71.jpg — no matching GT_IMG_71.mat
⚠️  Skipping

ValueError: num_samples should be a positive integer value, but got num_samples=0

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## **Introduction to Colab and Python**

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/examples/blob/master/courses/udacity_intro_to_tensorflow_for_deep_learning/l01c01_introduction_to_colab_and_python.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/examples/blob/master/courses/udacity_intro_to_tensorflow_for_deep_learning/l01c01_introduction_to_colab_and_python.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

Welcome to this Colab where you will get a quick introduction to the Python programming language and the environment used for the course's exercises: Colab.

Colab is a Python development environment that runs in the browser using Google Cloud.

For example, to print "Hello World", just hover the mouse over [ ] and press the play button to the upper left. Or press shift-enter to execute.

In [ ]:
print("Hello World")

## Functions, Conditionals, and Iteration
Let's create a Python function, and call it from a loop.

In [ ]:
def HelloWorldXY(x, y):
  if (x < 10):
    print("Hello World, x was < 10")
  elif (x < 20):
    print("Hello World, x was >= 10 but < 20")
  else:
    print("Hello World, x was >= 20")
  return x + y

for i in range(8, 25, 5):  # i=8, 13, 18, 23 (start, stop, step)
  print("--- Now running with i: {}".format(i))
  r = HelloWorldXY(i,i)
  print("Result from HelloWorld: {}".format(r))

In [ ]:
print(HelloWorldXY(1,2))

Easy, right?

If you want a loop starting at 0 to 2 (exclusive) you could do any of the following

In [ ]:
print("Iterate over the items. `range(2)` is like a list [0,1].")
for i in range(2):
  print(i)

print("Iterate over an actual list.")
for i in [0,1]:
  print(i)

print("While works")
i = 0
while i < 2:
  print(i)
  i += 1

In [ ]:
print("Python supports standard key words like continue and break")
while True:
  print("Entered while")
  break

## Numpy and lists
Python has lists built into the language.
However, we will use a library called numpy for this.
Numpy gives you lots of support functions that are useful when doing Machine Learning.

Here, you will also see an import statement. This statement makes the entire numpy package available and we can access those symbols using the abbreviated 'np' syntax.

In [ ]:
import numpy as np  # Make numpy available using np.

# Create a numpy array, and append an element
a = np.array(["Hello", "World"])
a = np.append(a, "!")
print("Current array: {}".format(a))
print("Printing each element")
for i in a:
  print(i)

print("\nPrinting each element and their index")
for i,e in enumerate(a):
  print("Index: {}, was: {}".format(i, e))

In [ ]:
print("\nShowing some basic math on arrays")
b = np.array([0,1,4,3,2])
print("Max: {}".format(np.max(b)))
print("Average: {}".format(np.average(b)))
print("Max index: {}".format(np.argmax(b)))

In [ ]:
print("\nYou can print the type of anything")
print("Type of b: {}, type of b[0]: {}".format(type(b), type(b[0])))

In [ ]:
print("\nUse numpy to create a [3,3] dimension array with random number")
c = np.random.rand(3, 3)
print(c)

In [ ]:
print("\nYou can print the dimensions of arrays")
print("Shape of a: {}".format(a.shape))
print("Shape of b: {}".format(b.shape))
print("Shape of c: {}".format(c.shape))
print("...Observe, Python uses both [0,1,2] and (0,1,2) to specify lists")

## Colab Specifics

Colab is a virtual machine you can access directly. To run commands at the VM's terminal, prefix the line with an exclamation point (!).


In [ ]:
print("\nDoing $ls on filesystem")
!ls -l
!pwd

In [ ]:
print("Install numpy")  # Just for test, numpy is actually preinstalled in all Colab instances
!pip install numpy

**Exercise**

Create a code cell underneath this text cell and add code to:


*   List the path of the current directory (pwd)
* Go to / (cd) and list the content (ls -l)

In [ ]:
!pwd
!cd /
!ls -l
print("Hello")

All usage of Colab in this course is completely free or charge. Even GPU usage is provided free of charge for some hours of usage every day.

**Using GPUs**
* Many of the exercises in the course executes more quickly by using GPU runtime: Runtime | Change runtime type | Hardware accelerator | GPU

**Some final words on Colab**
*   You execute each cell in order, you can edit & re-execute cells if you want
*   Sometimes, this could have unintended consequences. For example, if you add a dimension to an array and execute the cell multiple times, then the cells after may not work. If you encounter problem reset your environment:
  *   Runtime -> Restart runtime... Resets your Python shell
  *   Runtime -> Restart all runtimes... Will reset the Colab image, and get you back to a 100% clean environment
* You can also clear the output in the Colab by doing: Edit -> Clear all outputs
* Colabs in this course are loaded from GitHub. Save to your Google Drive if you want a copy with your code/output: File -> Save a copy in Drive...

**Learn More**
*   Check out [this](https://www.youtube.com/watch?v=inN8seMm7UI&list=PLQY2H8rRoyvwLbzbnKJ59NkZvQAW9wLbx&index=3) episode of #CodingTensorFlow, and don't forget to subscribe to the YouTube channel ;)


In [56]:
import os

train_image_path = os.path.join(train_path, 'images')
train_gt_path = os.path.join(train_path, 'ground_truth')
val_image_path = os.path.join(val_path, 'images')
val_gt_path = os.path.join(val_path, 'ground_truth')
test_image_path = os.path.join(test_path, 'images')
test_gt_path = os.path.join(test_path, 'ground_truth')

print(f"Train Images: {os.listdir(train_image_path)[:10]}") # Print first 10 files
print(f"Train Ground Truth: {os.listdir(train_gt_path)[:10]}") # Print first 10 files
print(f"Val Images: {os.listdir(val_image_path)[:10]}") # Print first 10 files
print(f"Val Ground Truth: {os.listdir(val_gt_path)[:10]}") # Print first 10 files
print(f"Test Images: {os.listdir(test_image_path)[:10]}") # Print first 10 files
print(f"Test Ground Truth: {os.listdir(test_gt_path)[:10]}") # Print first 10 files

Train Images: ['IMG_68.jpg', 'IMG_62.jpg', 'IMG_74.jpg', 'IMG_72.jpg', 'IMG_71.jpg', 'IMG_69.jpg', 'IMG_66.jpg', 'IMG_70.jpg', 'IMG_65.jpg', 'IMG_63.jpg']
Train Ground Truth: ['GT_IMG_81.mat', 'GT_IMG_80.mat', 'GT_IMG_74.mat', 'GT_IMG_73.mat', 'GT_IMG_79.mat', 'GT_IMG_77.mat', 'GT_IMG_76.mat', 'GT_IMG_75.mat', 'GT_IMG_78.mat', 'GT_IMG_72.mat']
Val Images: ['IMG_62.jpg', 'IMG_76.jpg', 'IMG_67.jpg', 'IMG_74.jpg', 'IMG_68.jpg', 'IMG_61.jpg', 'IMG_70.jpg', 'IMG_63.jpg', 'IMG_73.jpg', 'IMG_66.jpg']
Val Ground Truth: ['GT_IMG_81.mat', 'GT_IMG_79.mat', 'GT_IMG_77.mat', 'GT_IMG_80.mat', 'GT_IMG_74.mat', 'GT_IMG_69.mat', 'GT_IMG_76.mat', 'GT_IMG_75.mat', 'GT_IMG_72.mat', 'GT_IMG_78.mat']
Test Images: ['IMG_40.jpg', 'IMG_41.jpg', 'IMG_46.jpg', 'IMG_49.jpg', 'IMG_42.jpg', 'IMG_53.jpg', 'IMG_47.jpg', 'IMG_52.jpg', 'IMG_45.jpg', 'IMG_60.jpg']


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/CrowdDataset/test/ground_truth'